# 03. Mock Photon Comparison

In this Notebook we compare the mock photon propagation with the existing methods like the normal flow. To do that, we generate a line detector and a couple of events that we propagate using each propagator.

In [1]:
import pandas as pd

from ananke.configurations.detector import DetectorConfiguration
from ananke.services.detector import DetectorBuilderService
from olympus.event_generation.generators import get_generator
from olympus.event_generation.medium import MediumEstimationVariant, Medium
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from olympus.event_generation.photon_propagation.norm_flow_photons import NormalFlowPhotonPropagator
from olympus.configuration.generators import GenerationConfiguration
from olympus.configuration.photon_propagation import MockPhotonPropagatorConfiguration, NormalFlowPhotonPropagatorConfiguration

Let's define the detector next:

In [2]:
oms_per_line = 20
dist_z = 50  # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100  # m
pmts_per_module = 16
pmt_cath_area_r = 75e-3 / 2  # m
module_radius = 0.21  # m
efficiency = 0.42 # Christian S. Number

detector_configuration = DetectorConfiguration.parse_obj(
    {
        "string": {
            "module_number": oms_per_line,
            "module_distance": dist_z
        },
        "pmt": {
            "efficiency": efficiency,
            "noise_rate": dark_noise_rate,
            "area": pmt_cath_area_r
        },
        "module": {
            "radius": module_radius
        },
        "geometry": {
            "type": "single",
        },
        "seed": 31338
    }
)

detector_service = DetectorBuilderService()
det = detector_service.get(configuration=detector_configuration)

In [3]:
det.df[['module_location_x', 'module_location_y', 'module_location_z']].drop_duplicates()

,module_location_x,module_location_y,module_location_z
0,0.0,0.0,0.0
16,0.0,0.0,50.0
32,0.0,0.0,100.0
48,0.0,0.0,150.0
64,0.0,0.0,200.0
80,0.0,0.0,250.0
96,0.0,0.0,300.0
112,0.0,0.0,350.0
128,0.0,0.0,400.0
144,0.0,0.0,450.0


Next up we generate our events:

In [4]:
configuration = GenerationConfiguration.parse_obj({
    'generator': {
        'type': 'cascade',
        'spectrum': {
            'log_minimal_energy': 2.0,
            'log_maximal_energy': 5.5,
        }
    },
    'number_of_samples': 2
})

cascades_generator = get_generator(
    detector=det,
    configuration=configuration.generator
)

records = cascades_generator.generate_records(
    number_of_samples=2
)

sources = cascades_generator.propagate(records)

records.df.head()

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,energy,length,time,type,particle_id
0,42.151271,-3.693700,-221.257582,-0.186077,-0.861640,-0.472178,-6013544168474471955,838.859127,3000.0,0.0,cascade,11
1,25.837680,12.317769,597.338950,-0.161914,-0.749751,-0.641605,-6013543309481012755,423.830054,3000.0,0.0,cascade,11


In [5]:
sources.df.head()

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,time,number_of_photons,type
0,42.146339,-3.696871,-221.463745,-0.186077,-0.86164,-0.472178,-6013544168474471955,0.336900,265,cherenkov
1,42.038254,-3.858296,-221.550156,-0.186077,-0.86164,-0.472178,-6013544168474471955,1.004028,14680,cherenkov
2,42.093220,-4.064077,-221.619873,-0.186077,-0.86164,-0.472178,-6013544168474471955,1.671156,125478,cherenkov
3,41.849957,-4.267915,-221.575607,-0.186077,-0.86164,-0.472178,-6013544168474471955,2.338284,492842,cherenkov
4,41.891029,-4.522918,-221.550034,-0.186077,-0.86164,-0.472178,-6013544168474471955,3.005413,1278135,cherenkov


Now we need our photon propagators

In [6]:
mock_photon_propagator_configuration = MockPhotonPropagatorConfiguration(resolution=18000)

mock_photon_propagator = MockPhotonPropagator(
    detector=det,
    configuration=mock_photon_propagator_configuration
)

normal_photon_propagator_configuration = NormalFlowPhotonPropagatorConfiguration(
    shape_model_path="../../hyperion/data/photon_arrival_time_nflow_params.pickle",
    counts_model_path="../../hyperion/data/photon_arrival_time_counts_params.pickle"
)

normal_photon_propagator = NormalFlowPhotonPropagator(
    detector=det,
    configuration=normal_photon_propagator_configuration
)

Now lets Propagate:

In [9]:
mock_hits = mock_photon_propagator.propagate(records, sources, use_multiprocessing=False)

mock_hits.df.head()

,time,string_id,module_id,pmt_id,record_id,type
0,8.684685,0,9,1,-6013543309481012755,cascade
1,14.342976,0,9,1,-6013543309481012755,cascade
2,9.634443,0,9,1,-6013543309481012755,cascade
3,10.676672,0,9,1,-6013543309481012755,cascade
4,16.904922,0,9,1,-6013543309481012755,cascade


And doing the same with the normal photons.

In [10]:
normal_hits = normal_photon_propagator.propagate(records, sources)

normal_hits.df.head()

,time,string_id,module_id,pmt_id,record_id,type
0,10.878100,0,8,2,-6013543309481012755,cascade
0,59.620186,0,8,9,-6013543309481012755,cascade
0,17.784218,0,8,10,-6013543309481012755,cascade
0,13.448198,0,8,14,-6013543309481012755,cascade
0,13.620386,0,9,1,-6013543309481012755,cascade


## Comparison of the photon propagators

Now that we have all the hits we want we can compare the following cases:

1. number of hits per module
2. hit arrival times


### Number of hits per module

Let's have a look at the mock propagation:

In [11]:
aggregated_mock_hits = mock_hits.df.set_index(['string_id', 'module_id'])
aggregated_mock_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id  type
string_id module_id                               
0         9            47      47         47    47
          10          864     864        864   864
          11         3121    3121       3121  3121
          12          703     703        703   703

In [12]:
aggregated_normal_hits = normal_hits.df.set_index(['string_id', 'module_id'])
aggregated_normal_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id  type
string_id module_id                               
0         8             4       4          4     4
          9             8       8          8     8
          10           71      71         71    71
          11          500     500        500   500
          12          611     611        611   611

In [ ]:
len(mock_hits)

In [ ]:
len(normal_hits)